In [34]:
import numpy as np
from scipy import sparse
import warnings

warnings.filterwarnings("ignore", category=sparse.SparseEfficiencyWarning)
np.set_printoptions(linewidth=150)

1. 使用雅可比方法求解稀疏系统，精确到小数点后6位(用无穷范数表示的前向误差)，其中$n=100$和$n=100 000$. 正确解是 $[1,\dots,1]$. 报告所需要的步数和后向误差，系统如下:
$$
\begin{bmatrix}
 3 & -1 \\
-1 &  3 & -1 \\
&\ddots &\ddots &\ddots \\
&&-1&3&-1 \\
&&&-1&3 \\
\end{bmatrix}
\begin{bmatrix}
x_1\\
\\
\vdots\\
\\
x_n
\end{bmatrix}
=
\begin{bmatrix}
2\\
1\\
\vdots\\
1\\
2
\end{bmatrix}
$$

In [6]:
def Jacobi(n, b, toleranceBit, D, DInverse, U, L, answer, maxIter=1000):
    """
    param: n: Dimension.
    param: D: matrix of principle diagonal.
    param: DInverse: Inverse matrix of D.
    param: U: Matrix with elements above principle diagonal.
    param: L: Matrix with elements below principle diagonal.
    param: b: Constant array.
    """
    x = np.zeros(n)
    tolerance = 10**(-toleranceBit)
    count = 0
    FE = float('inf')

    while(FE > tolerance and count < maxIter):
        count += 1
        x = DInverse@(b-(L+U)@x)
        FE = np.linalg.norm(x-answer, np.inf)

    
    print("Jacobi iteration amount: ", count)
    print("    n: ", n)
    A = D+U+L
    print("   BE: ", np.linalg.norm(A@x-b, np.inf))
    print("   FE: ", FE)
    print()
    return x

def Solver(n):
    b = np.ones(n)
    b[0] = 2.
    b[-1] = 2.

    D = sparse.lil_matrix((n, n))
    DInverse = D.copy()
    U = D.copy()
    L = D.copy()

    D.setdiag(3)
    DInverse.setdiag(1/3)
    U.setdiag(-1, k=1)
    L.setdiag(-1, k=-1)

    x = Jacobi(n, b, 6, D, DInverse, U, L, answer=np.ones(n))

Solver(100)
Solver(10000)
Solver(100000)


Jacobi iteration amount:  35
    n:  100
   BE:  6.867832086365766e-07
   FE:  6.867614883443451e-07

Jacobi iteration amount:  35
    n:  10000
   BE:  6.867832086365766e-07
   FE:  6.867614883443451e-07

Jacobi iteration amount:  35
    n:  100000
   BE:  6.867832086365766e-07
   FE:  6.867614883443451e-07




2. 使用雅可比方法求解稀疏系统，精确到小数点后3位(用无穷范数表示的前向误差)，其中n=100. 正确解是 $[1,-1,1,-1,\dots,1,-1]$.报告所需要的步数和后向误差，系统如下:
$$
\begin{bmatrix}
 2 & 1 \\
1 &  2 & 1 \\
&\ddots &\ddots &\ddots \\
&&1&2&1 \\
&&&1&2 \\
\end{bmatrix}
\begin{bmatrix}
x_1\\
\\
\vdots\\
\\
x_n
\end{bmatrix}
=
\begin{bmatrix}
1\\
0\\
\vdots\\
0\\
-1
\end{bmatrix}
$$

In [11]:
n = 100
b = np.zeros(n)
b[0] = 1.
b[-1] = -1.

D = sparse.lil_matrix((n, n))
DInverse = D.copy()
U = D.copy()
L = D.copy()

D.setdiag(2.)
DInverse.setdiag(0.5)
U.setdiag(1, k=1)
L.setdiag(1, k=-1)

answer = np.ones(n)
answer[1::2] = -1
x = Jacobi(n, b, 3, D, DInverse, U, L, answer, maxIter=100000)

Jacobi iteration amount:  14776
    n:  100
   BE:  9.674122976033317e-07
   FE:  0.00099997610349023



3. 重写程序2.2进行高斯-塞德尔迭代，求解例2.24（如下）中的问题验证你的工作.
$$
\begin{bmatrix}
 3&-1& 0& 0& 0&\frac12\\
-1& 3&-1& 0&\frac12& 0\\
 0&-1& 3&-1& 0& 0\\
 0& 0&-1& 3&-1& 0\\
 0&\frac12& 0&-1& 3&-1\\
\frac12& 0& 0& 0&-1& 3\\
\end{bmatrix}
\begin{bmatrix}
u_1\\
u_2\\
u_3\\
u_4\\
u_5\\
u_6\\
\end{bmatrix}
=
\begin{bmatrix}
\frac52\\
\frac32\\
1\\
1\\
\frac32\\
\frac52\\
\end{bmatrix}
\\
解 x=[1,1,1,1,1,1]
$$

In [32]:
def Gauss_Seidel(n, b, D, U, L, answer, maxIter=1000, toleranceBit=6):
    x = np.zeros(n)
    tolerance = 10**(-toleranceBit)
    count = 0
    FE = float('inf')

    if type(D) is np.ndarray:
        LDInverse = np.linalg.inv(L+D)
    else:
        LDInverse = sparse.linalg.inv(L+D)

    while(FE > tolerance and count < maxIter):
        count += 1
        x = LDInverse@(b-U@x)
        FE = np.linalg.norm(x-answer, np.inf)

    print("Gauss_Seidel iteration amount: ", count)
    print("    n: ", n)
    print("   BE: ", np.linalg.norm((D+U+L)@x-b, np.inf))
    print("   FE: ", FE)
    print()
    return x

b = np.array([5/2, 3/2, 1, 1, 3/2, 5/2])
D = np.eye(6)*3.
U = np.zeros((6, 6))
np.fill_diagonal(U[:-1, 1:], -1)
U[0, 5] = 0.5
U[1, 4] = 0.5
L = np.zeros((6, 6))
np.fill_diagonal(L[1:, :-1], -1)
L[4, 1] = 0.5
L[5, 0] = 0.5

answer = np.ones(6)
x = Gauss_Seidel(6, b, D, U, L, answer, maxIter=1000)
print(x)

Gauss_Seidel iteration amount:  15
    n:  6
   BE:  8.667896986835899e-07
   FE:  6.134616969966089e-07

[1.0000004  1.00000061 1.00000061 1.00000035 1.00000003 0.99999994]


4. 重写程序2.2进行SOR.使用 $\omega=1.1$，再次验证例 2.24.


In [26]:
def SOR(n, b, D, U, L, answer, omega, maxIter=1000, toleranceBit=6):
    "Successive Over-Relaxation"
    x = np.zeros(n)
    tolerance = 10**(-toleranceBit)
    count = 0
    FE = float('inf')

    if type(D) is np.ndarray:
        LDInverse = np.linalg.inv(omega*L+D)
    else:
        LDInverse = sparse.linalg.inv(omega*L+D)

    while(FE > tolerance and count < maxIter):
        count += 1
        x = LDInverse@(omega*b+((1-omega)*D-omega*U)@x)
        FE = np.linalg.norm(x-answer, np.inf)

    print("SOR iteration amount: ", count)
    print("    n: ", n)
    print("   BE: ", np.linalg.norm((D+U+L)@x-b, np.inf))
    print("   FE: ", FE)
    print()
    return x

x = SOR(6, b, D, U, L, answer, 1.1, maxIter=1000)
print(x)

SOR iteration amount:  12
    n:  6
   BE:  2.3307373684389177e-06
   FE:  9.825712352640181e-07

[1.00000098 1.00000048 0.99999956 0.99999922 0.99999942 0.99999972]


5. 执行编程问题1中的步骤，n=100.
    1. 高斯-塞德尔方法
    2. SOR，$\omega=1.2$.

In [35]:
n = 100
b = np.ones(n)
b[0] = 2.
b[-1] = 2.

D = sparse.lil_matrix((n, n))
DInverse = D.copy()
U = D.copy()
L = D.copy()

D.setdiag(3)
U.setdiag(-1, k=1)
L.setdiag(-1, k=-1)

answer = np.ones(n)
x = Gauss_Seidel(n, b, D, U, L, answer)
x = SOR(n, b, D, U, L, answer, 1.2)

Gauss_Seidel iteration amount:  20
    n:  100
   BE:  9.564705890641179e-07
   FE:  9.5367431640625e-07

SOR iteration amount:  16
    n:  100
   BE:  1.5515907685337282e-06
   FE:  4.062661562720393e-07



6. 执行编程问题2中的步骤，
    1. 高斯-塞德尔方法
    2. SOR，$\omega=1.5$.

In [37]:
n = 100
b = np.zeros(n)
b[0] = 1.
b[-1] = -1.

D = sparse.lil_matrix((n, n))
U = D.copy()
L = D.copy()

D.setdiag(2.)
U.setdiag(1, k=1)
L.setdiag(1, k=-1)

answer = np.ones(n)
answer[1::2] = -1

x = Gauss_Seidel(n, b, D, U, L, answer, maxIter=100000)
x = SOR(n, b, D, U, L, answer, 1.5, maxIter=100000)

Gauss_Seidel iteration amount:  14528
    n:  100
   BE:  9.675009682297286e-10
   FE:  9.993421266063507e-07

SOR iteration amount:  4835
    n:  100
   BE:  9.733464034766826e-10
   FE:  9.995389900208096e-07



7. 使用编程问题3的程序，确定形如(2.38)的系统，使用高斯-塞德尔方法在1秒内所能够精确求解的系统最大规模，报告对于不同的n所需的时间，以及前向误差.

In [ ]:
# 形如2.38?